# Baseline-решение

По мотивам ноутбука https://www.kaggle.com/code/hardtype/parsing-news-from-rbc-lenta-ru

## 1. Парсим новости с сайта Lenta.ru

In [ ]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [ ]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
# from IPython import display

In [ ]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [ ]:
# Задаем тут параметры
query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0" # topic = тематика новости
dateFrom = '2023-01-01'
dateTo = "2023-12-30"

param_dict = {'query'     : query,
              'from'      : str(offset),
              'size'      : str(size),
              'dateFrom'  : dateFrom,
              'dateTo'    : dateTo,
              'sort'      : sort,
              'title_only': title_only,
              'type'      : material,
              'bloc'      : bloc,
              'domain'    : domain}

print("param_dict:", param_dict)

param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2023-01-01', 'dateTo': '2023-12-30', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '0', 'domain': '1'}


In [ ]:
# Тоже будем собирать итеративно, правда можно ставить time_step побольше, т.к.
# больше лимит на запрос статей. И Работает быстрее :)

parser = lentaRu_parser()

tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Finish
10000


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1363803,https://lenta.ru/news/2023/01/01/exponenta/,Ким Чен Ын пообещал нарастить производство яде...,1672531825,1672531825,1,1,0,0,2,[1],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672531825,Фото: ЦТАК / Reuters Марина Совина Лидер КНДР ...,Ким Чен Ын пообещал нарастить производство яде...,Фото: ЦТАК / Reuters Марина Совина Лидер ... я...
1,1363805,https://lenta.ru/news/2023/01/01/yaroslavl/,В российском городе пропал ребенок,1672532081,1672532081,1,1,0,0,1,[4],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672532081,Фото: Кирилл Шипицин / РИА Новости Марина Сови...,В российском городе пропал ребенок,Фото: Кирилл Шипицин / РИА Новости ... мальчик...
2,1363807,https://lenta.ru/news/2023/01/01/alco/,Россиянам рассказали о влиянии алкоголя на сон,1672533004,1672533004,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672533004,Фото: Pixabay Марина Совина Терапевт Ирина Анд...,Россиянам рассказали о влиянии алкоголя на сон,Фото: Pixabay Марина Совина Терапевт ... употр...
3,1363808,https://lenta.ru/news/2023/01/01/ded_moroz/,Подсчитана пенсия Деда Мороза,1672533251,1672533252,1,1,0,1,1,[2],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672533251,Фото: Илья Наймушин/ РИА Новости Марина Совина...,Подсчитана пенсия Деда Мороза,Фото: Илья Наймушин/ РИА Новости Марина ... Ги...
4,1363806,https://lenta.ru/news/2023/01/01/anomalia_/,Климатолог предупредил о возможных погодных ан...,1672533531,1672533531,1,1,0,0,12,[281],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672533531,Фото: Komsomolskaya Pravda / Global Look Press...,Климатолог предупредил о возможных погодных ан...,Фото: Komsomolskaya Pravda / Global Look ... К...


In [ ]:
tbl.to_csv("Lenta_sample.csv", index=False)

In [ ]:
# tbl = pd.read_csv("Lenta_sample.csv")

In [ ]:
tbl.shape

(10000, 16)

In [ ]:
tbl['bloc'].value_counts(normalize=True)

1     0.1891
2     0.1833
3     0.1083
4     0.0858
8     0.0687
37    0.0628
5     0.0449
12    0.0377
7     0.0366
47    0.0365
6     0.0332
48    0.0296
87    0.0280
9     0.0240
0     0.0153
86    0.0124
49    0.0036
11    0.0002
Name: bloc, dtype: float64

Найдем соответствие между кодом блока, его названием и кодом в соревновании:

* 1 - Россия - 0
* 37 - Силовые структуры - 2
* 3 - Бывший СССР - 3
* 4 - Экономика - 1
* 5 - Наука и техника - 8
* 8 - Спорт - 4
* 48 - Туризм - 7
* 87 - Здоровье - 5

In [ ]:
tbl[tbl.bloc == 3].iloc[0]

docid                                                    1363858
url                       https://lenta.ru/news/2023/01/01/kule/
title          На Украине заявили о поражении объекта инфраст...
modified                                              1672557480
lastmodtime                                           1672558031
type                                                           1
domain                                                         1
status                                                         0
part                                                           0
bloc                                                           3
tags                                                        [55]
image_url      https://icdn.lenta.ru/images/2023/01/01/10/202...
pubdate                                               1672557480
text           Фото: Gleb Garanich / Reuters Варвара Кошечкин...
rightcol       На Украине заявили о поражении объекта инфраст...
snippet        Фото: Gleb

In [ ]:
tbl = tbl[tbl.bloc.isin([1, 37, 3, 4, 5, 8, 48, 87])]

TagsMap = {1 : 0, 3 : 3, 4 : 1, 5 : 8, 8 : 4, 37 : 2, 48 : 7, 87 : 5}

tbl['topic'] = tbl['bloc'].map(TagsMap)

<ipython-input-36-5f5bf2ec18e9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl['topic'] = tbl['bloc'].map(TagsMap)


In [ ]:
tbl.shape

(6172, 17)

In [ ]:
tbl['topic'].value_counts(normalize=True) # можно сверить с распределением меток классов в соревновании

0    0.306384
3    0.175470
1    0.139015
4    0.111309
2    0.101750
8    0.072748
7    0.047959
5    0.045366
Name: topic, dtype: float64

## 2. Машинное обучение

Загружаем данные и обучаем модель на разбиении трейн-тест

In [ ]:
tbl_new = tbl[~tbl.text.isna()]

print(len(tbl), len(tbl_new))

6172 6018


In [ ]:
X = tbl_new[['text']]
y = tbl_new['topic']

X.shape

(6018, 1)

In [ ]:
# использовать "вероятностные модели"

# class_weight = "balanced"

# в обучающих данных взять поровну новостей каждого класса

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((4513, 1), (1505, 1))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = CountVectorizer() # подбор гиперпараметров очень помогает
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(4513, 83231)
              precision    recall  f1-score   support

           0       0.83      0.94      0.88       472
           1       0.95      0.92      0.93       209
           2       0.93      0.95      0.94       151
           3       0.91      0.85      0.87       271
           4       0.99      0.99      0.99       137
           5       1.00      0.86      0.93        66
           7       0.98      0.86      0.92        73
           8       0.96      0.81      0.88       126

    accuracy                           0.91      1505
   macro avg       0.94      0.90      0.92      1505
weighted avg       0.91      0.91      0.91      1505



Загружаем тестовые данные, обучаем итоговую модель и делаем прогноз.

In [ ]:
Test = pd.read_csv("test_news.csv")
Test

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = CountVectorizer()
vec.fit(X['text'])

bow = vec.transform(X['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(Test['content'])

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y)
pred = clf.predict(bow_test)

In [ ]:
pred[:10], len(pred)

(array([0, 0, 4, 0, 4, 3, 2, 3, 0, 3]), 26275)

Сохраняем прогноз в файл.

In [ ]:
subm = pd.read_csv("base_submission_news.csv")
subm.head()

,topic,index
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [ ]:
subm['topic'] = pred

subm.to_csv("bow_logreg_lenta.csv", index=False)

In [ ]:
subm['topic'].value_counts(normalize=True)

0    0.484681
1    0.151170
3    0.100133
4    0.083007
2    0.081979
8    0.048754
7    0.030676
5    0.019600
Name: topic, dtype: float64